In [1]:
!pip install torch transformers


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
dataset_path = "/content/drive/MyDrive/Major_Project_Final-Yr/Dataset (All) /destination_file.txt"
with open(dataset_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    dataset = [line.strip() for line in lines]


In [6]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [7]:
tokenizer.pad_token = tokenizer.eos_token


In [8]:
# Tokenize the dataset
# tokenized_dataset = [tokenizer.encode_plus(text, add_special_tokens=True, max_length=512, truncation=True, padding="max_length", return_attention_mask=True, return_tensors='pt') for text in dataset]
#remove
tokenized_dataset = [tokenizer.encode_plus(
    text,
    add_special_tokens=True,
    max_length=512,  # Adjust based on your needs
    truncation=True,
    padding="max_length",
    return_attention_mask=True,
    return_tensors='pt'
) for text in dataset]


In [9]:
# Preparing My DataLoader
from torch.utils.data import Dataset, DataLoader

class GPT2Dataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings)

    def __getitem__(self, idx):
        return self.encodings[idx]

# Create the dataset and dataloader
dataset = GPT2Dataset(tokenized_dataset)
loader = DataLoader(dataset, batch_size=4, shuffle=True)



In [10]:
# Fine Tuning the Model
from transformers import GPT2LMHeadModel, AdamW
from torch.nn import CrossEntropyLoss
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GPT2LMHeadModel.from_pretrained('gpt2')
model.to(device)
model.train()

optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 1
for epoch in range(num_epochs):
    for batch in loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = input_ids.clone()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch} loss: {loss.item()}")


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 0 loss: 4.457061767578125


In [11]:
model.save_pretrained('/content/drive/MyDrive/Major_Project_Final-Yr/myGpt')
tokenizer.save_pretrained('/content/drive/MyDrive/Major_Project_Final-Yr/myGpt')


('/content/drive/MyDrive/Major_Project_Final-Yr/myGpt/tokenizer_config.json',
 '/content/drive/MyDrive/Major_Project_Final-Yr/myGpt/special_tokens_map.json',
 '/content/drive/MyDrive/Major_Project_Final-Yr/myGpt/vocab.json',
 '/content/drive/MyDrive/Major_Project_Final-Yr/myGpt/merges.txt',
 '/content/drive/MyDrive/Major_Project_Final-Yr/myGpt/added_tokens.json')